# **Exercise: Numba Basics**

The function below is a naive sum function that sums all the elements of a given array. Let’s get a numba version of this code running.

In [ ]:
import numpy
numpy.random.seed(1)

In [ ]:
#Array summation
def summation_array(input):
    J, I = input.shape
    sum = 0
    for j in range(J):
        for i in range(I):
            sum += input[j, i]   
    return sum

In [ ]:
array = numpy.random.random((300, 300))

In [ ]:
summation_array(array)

Let's get started to write numba version.

##### **As a function call**


In [ ]:
from numba import jit

In [ ]:
summation_array_numba = jit()(summation_array)

In [ ]:
summation_array_numba(array)

##### **As a decorater (more commonly)**

In [ ]:
@jit
def summation_array_numba_dec(inp):
    I, J = inp.shape
    mysum = 0
    for i in range(I):
        for j in range(J):
            mysum += inp[i, j]
    return mysum

In [ ]:
summation_array_numba_dec(array)

##### **Benchmarking**

Compare NumPy and Numba using %timeit

In [ ]:
time_numpy = %timeit -o summation_array(array)

In [ ]:
time_jitted = %timeit -o summation_array_numba(array)

In [ ]:
time_numpy.best / time_jitted.best

In [ ]:
time_dec = %timeit -o summation_array_numba_dec(array)

##### **Question: When does Numba compile codes?**

The first time you call the function.

# **Exercise: Writing CUDA kernels**

We’ll work an example function that determines if a point is in the mandelbrot set or not. Below statement is a pure Python implementation of a function. 

## **Python Implemtation**

In [ ]:
from __future__ import print_function, division, absolute_import

from timeit import default_timer as timer
from matplotlib.pylab import imshow, show
import numpy as np


def mandelbrot(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255

# The image loop
# This is the calling function. 
# It takes a 2D array called image, and calls the mandelbrot function for every pixel in that image.

def create_fractal(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = mandelbrot(real, imag, iters)
            image[y, x] = color


image = np.zeros((500 * 10, 750 * 10), dtype=np.uint8)

s = timer()
create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20)
e = timer()

print("Execution time: %f seconds" % (e - s))

imshow(image)
show()

## **Task**

Modify the example by compiling:
- @jit decorator, to run as native code on the CPU
- @cuda.jit decorator, to run on the GPU

**Note:**  CUDA kernels are compiled using the numba.cuda.jit decorator (not to be confused with the numba.jit decorator for the CPU)

### **Mandel JIT Implementation**

If we want to use Numba to compile this function for the CPU, then we do two things: first, we
import the jit decorator from Numba, then we “decorate” the function with it.

In [ ]:
from __future__ import print_function, division, absolute_import

from timeit import default_timer as timer
from matplotlib.pylab import imshow, show
import numpy as np

from numba import jit

# Turning Pure python code into compiled native code
@jit
def mandel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255

# Compile the outer function for the CPU
# Compiled, jitted function can call another one.
@jit
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x] = color

    return image


image = np.zeros((500 * 10, 750 * 10), dtype=np.uint8)

s = timer()
create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20)
e = timer()
print("Execution time: %f seconds" % (e - s))

imshow(image)
show()

**Mandel CUDA JIT Implementation**

We use the cuda.jit decorator to compile a function and to run on a GPU. We will also pass the **device=True** keyword argument.

The original code contained two for loops: one over the width of the image and one over the height.

To parallelise those loops across threads, the loop structure is flattened so that each pixel is assigned to one thread.

Numba provides a convenience function called grid, which we’re using here. You call it with the number of dimensions of the grid, and it returns N linear indices. You can think of this as a shorthand for computations with the block dimensions and index, and the thread index.

The grid can often be a little bigger than the image, so we add a guard to make sure that only threads within the image bounds do the computation.

Those are all the changes made to make this into a global kernel function. Conceptually they’re very similar to CUDA C, and only the language is a little bit different.

In [ ]:
from __future__ import print_function, division, absolute_import

from timeit import default_timer as timer
from matplotlib.pylab import imshow, show
import numpy as np

from numba import cuda


@cuda.jit(device=True)
def mandel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i

    return 255


@cuda.jit
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    x, y = cuda.grid(2)

    if x < width and y < height:
        real = min_x + x * pixel_size_x
        imag = min_y + y * pixel_size_y
        color = mandel(real, imag, iters)
        image[y, x] = color

# we create a grid of 32x32 blocks that’s just big enough to contain the image, and launch with
that configuration.
width = 15000
height = 10000
image = np.zeros((height, width), dtype=np.uint8)

pixels = width * height
nthreads = 32
nblocksy = (height // nthreads) + 1
nblocksx = (width // nthreads) + 1
s = timer()

create_fractal[(nblocksx, nblocksy), (nthreads, nthreads)](-2.0, 1.0, -1.0, 1.0, image, 20)

e = timer()
print("Execution time: %f seconds" % (e - s))

imshow(image)
show()

**What's next?**

Can you also rewrite the mandel function using @vectorize for the CUDA target?

**Mandel Vectorize Implementation**

In [ ]:
from __future__ import print_function, division, absolute_import


from timeit import default_timer as timer

from matplotlib.pylab import imshow, show

from numba import vectorize
import numpy as np

sig = "uint8(uint32, f4, f4, f4, f4, uint32, uint32, uint32)"


@vectorize([sig], target="cuda")
def mandel(tid, min_x, max_x, min_y, max_y, width, height, iters):
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height

    x = tid % width
    y = tid / width

    real = min_x + x * pixel_size_x
    imag = min_y + y * pixel_size_y

    c = complex(real, imag)
    z = 0.0j

    for i in range(iters):
        z = z * z + c
        if (z.real * z.real + z.imag * z.imag) >= 4:
            return i
    return 255


def create_fractal(min_x, max_x, min_y, max_y, width, height, iters):
    tids = np.arange(width * height, dtype=np.uint32)
    return mandel(
        tids,
        np.float32(min_x),
        np.float32(max_x),
        np.float32(min_y),
        np.float32(max_y),
        np.uint32(height),
        np.uint32(width),
        np.uint32(iters),
    )


def main():
    width = 500 * 10
    height = 750 * 10

    ts = timer()
    pixels = create_fractal(-2.0, 1.0, -1.0, 1.0, width, height, 20)
    te = timer()
    print("Execution time: %f" % (te - ts))

    image = pixels.reshape(width, height)
    imshow(image)
    show()


if __name__ == "__main__":
    main()